In [13]:
from pandas_profiling import ProfileReport

from sklearn import metrics
from analyse_dataset import filling_rate, analyse, val_tri
from heatmap import heatmap, corrplot
from sklearn import decomposition
from sklearn import preprocessing
import math
import folium
import geopandas as gpd
import seaborn as sns
import os
import glob
import pandas as pd
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [14]:
pd.set_option('display.max_columns', 120)
pd.set_option('display.max_rows', 100)

In [15]:
data_all = pd.read_csv('application_train.csv',sep='\t',index_col=1)
data_all.drop(columns=['Unnamed: 0'],inplace= True)
data_all.head()

,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,OWN_CAR_AGE,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,DAYS_BIRTH_1,DAYS_EMPLOYED_1,DAYS_REGISTRATION_1,DAYS_ID_PUBLISH_1,REGION_RATING_CLIENT_MEAN,WEEKDAY_APPR_PROCESS_START_cos,WEEKDAY_APPR_PROCESS_START_sin,HOUR_APPR_PROCESS_START_cos,HOUR_APPR_PROCESS_START_sin,EXT_SOURCE_AVG,DAYS_LAST_PHONE_CHANGE_1,number_documents_provided
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,0.0,1,0,1,0,Laborers,1.0,0,0,0,0,0,0,Business Entity Type 3,0.0247,0.0369,0.9722,0.6192,0.0143,0.00,0.0690,0.0833,0.1250,0.0369,0.0202,0.0190,0.0000,0.0000,0.0252,0.0383,0.9722,0.6341,0.0144,0.0000,0.0690,0.0833,0.1250,0.0377,0.022,0.0198,0.0,0.0,0.0250,0.0369,0.9722,0.6243,0.0144,0.00,0.0690,0.0833,0.1250,0.0375,0.0205,0.0193,0.0000,0.00,reg oper account,block of flats,0.0149,"Stone, brick",No,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,25.920548,1.745205,9.994521,5.808219,2.0,-0.900969,0.433884,-0.917211,0.398401,0.161787,3.106849,1
100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,0.0,1,0,1,0,Core staff,2.0,0,0,0,0,0,0,School,0.0959,0.0529,0.9851,0.7960,0.0605,0.08,0.0345,0.2917,0.3333,0.0130,0.0773,0.0549,0.0039,0.0098,0.0924,0.0538,0.9851,0.8040,0.0497,0.0806,0.0345,0.2917,0.3333,0.0128,0.079,0.0554,0.0,0.0,0.0968,0.0529,0.9851,0.7987,0.0608,0.08,0.0345,0.2917,0.3333,0.0132,0.0787,0.0558,0.0039,0.01,reg oper account,block of flats,0.0714,Block,No,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,45.931507,3.254795,3.249315,0.797260,1.0,0.623490,0.781831,-0.990686,0.136167,0.466757,2.268493,1
100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,26.0,1,1,1,0,Laborers,1.0,0,0,0,0,0,0,Government,0.0000,0.0000,0.0000,0.0000,0.0000,0.00,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000,0.0000,0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.00,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.00,None,None,0.0000,None,None,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,52.180822,0.616438,11.671233,6.934247,2.0,0.623490,0.781831,-0.775711,0.631088,0.642739,2

In [16]:
data_0 = data_all.loc[data_all.TARGET==0].sample(n=182419,random_state=42)
data_1 = data_all.loc[data_all.TARGET==1]
data= pd.concat([data_0,data_1],axis=0)

In [17]:
data.shape

(207222, 99)

In [18]:
display(len(data.loc[data.TARGET==0]),len(data.loc[data.TARGET==1]))

182419

24803

In [19]:
target = data.TARGET
data.drop(columns=['TARGET'], inplace=True)

In [20]:
data.FLAG_EMP_PHONE.replace(to_replace=[0,1],value=['0','1'],inplace=True)
data.FLAG_WORK_PHONE.replace(to_replace=[0,1],value=['0','1'],inplace=True)
data.FLAG_PHONE.replace(to_replace=[0,1],value=['0','1'],inplace=True)
data.FLAG_EMAIL.replace(to_replace=[0,1],value=['0','1'],inplace=True)
data.REG_REGION_NOT_LIVE_REGION.replace(to_replace=[0,1],value=['0','1'],inplace=True)
data.REG_REGION_NOT_WORK_REGION.replace(to_replace=[0,1],value=['0','1'],inplace=True)
data.LIVE_REGION_NOT_WORK_REGION.replace(to_replace=[0,1],value=['0','1'],inplace=True)
data.REG_CITY_NOT_LIVE_CITY.replace(to_replace=[0,1],value=['0','1'],inplace=True)
data.REG_CITY_NOT_WORK_CITY.replace(to_replace=[0,1],value=['0','1'],inplace=True)
data.LIVE_CITY_NOT_WORK_CITY.replace(to_replace=[0,1],value=['0','1'],inplace=True)

In [21]:
data.describe()

,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,OWN_CAR_AGE,CNT_FAM_MEMBERS,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,DAYS_BIRTH_1,DAYS_EMPLOYED_1,DAYS_REGISTRATION_1,DAYS_ID_PUBLISH_1,REGION_RATING_CLIENT_MEAN,WEEKDAY_APPR_PROCESS_START_cos,WEEKDAY_APPR_PROCESS_START_sin,HOUR_APPR_PROCESS_START_cos,HOUR_APPR_PROCESS_START_sin,EXT_SOURCE_AVG,DAYS_LAST_PHONE_CHANGE_1,number_documents_provided
count,207222.000000,2.072220e+05,2.072220e+05,207222.000000,2.072220e+05,207222.000000,207222.000000,207222.000000,207222.000000,207222.000000,207222.000000,207222.000000,207222.000000,207222.000000,207222.000000,207222.000000,207222.000000,207222.000000,207222.000000,207222.000000,207222.000000,207222.000000,207222.000000,207222.000000,207222.000000,207222.000000,207222.000000,207222.000000,207222.000000,207222.000000,207222.000000,207222.000000,207222.000000,207222.000000,207222.000000,207222.000000,207222.000000,207222.000000,207222.000000,207222.000000,207222.00000,207222.000000,207222.000000,207222.000000,207222.000000,207222.000000,207222.000000,207222.000000,207222.000000,207222.000000,207222.000000,207222.000000,207222.000000,207222.000000,207222.000000,207222.000000,207222.000000,207222.000000,207222.000000,207222.000000,207222.000000,207222.000000,207222.000000,207222.000000,207222.000000,207222.000000,207222.000000,207222.000000,207222.000000,207222.000000,207222.000000,207222.000000,207222.000000
mean,0.417822,1.680998e+05,5.974776e+05,27087.909323,5.364002e+05,0.020788,4.118969,2.152831,0.057349,0.036304,0.497170,0.249869,0.013274,0.036594,0.073793,0.112669,0.073873,0.026709,0.031547,0.053081,0.002658,0.012628,0.055788,0.035951,0.496800,0.252255,0.012675,0.034564,0.071572,0.110713,0.072667,0.026155,0.033067,0.052414,0.002452,0.012033,0.057562,0.036118,0.497175,0.250964,0.01327,0.036205,0.073544,0.112502,0.073799,0.027044,0.031925,0.053687,0.002616,0.012570,0.052617,1.416254,0.144806,1.399436,0.101056,0.005699,0.006095,0.029900,0.229483,0.228822,1.783372,43.815471,5.316719,13.598337,8.170658,2.047227,-0.152084,0.055180,-0.660185,-0.100687,0.504583,2.620496,0.930799
std,0.720867,9.889799e+04,4.004940e+05,14423.437799,3.675706e+05,0.013755,9.070084,0.910106,0.095886,0.068168,0.490585,0.360262,0.046156,0.099801,0.102712,0.152241,0.141310,0.060746,0.069860,0.094621,0.026211,0.048747,0.094820,0.068874,0.490611,0.363265,0.045022,0.097544,0.101555,0.150379,0.139908,0.060664,0.073613,0.095057,0.025596,0.048904,0.096496,0.067938,0.490642,0.361662,0.04624,0.099559,0.102690,0.152343,0.141419,0.061466,0.070705,0.095933,0.026147,0.049055,0.092643,2.318356,0.445904,2.298609,0.362025,0.079335,0.105429,0.193299,0.850255,0.812633,1.771269,11.954601,6.271110,9.629588,4.137856,0.500030,0.651130,0.741526,0.392897,0.632181,0.152030,2.258925,0.342016
min,0.000000,2.565000e+04,4.500000e+04,1615.500000,4.050000e+04,0.000290,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,

In [22]:
data.select_dtypes('object')

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,FONDKAPREMONT_MODE,HOUSETYPE_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,
371535,Cash loans,F,Y,N,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,1,0,1,1,Cooking staff,0,0,0,0,0,0,Industry: type 4,reg oper account,block of flats,Panel,No
269653,Cash loans,F,N,Y,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,1,0,1,0,High skill tech staff,0,0,0,0,0,0,Self-employed,None,None,None,None
184406,Cash loans,M,Y,Y,Children,Working,Lower secondary,Married,House / apartment,1,0,0,0,Drivers,0,0,0,1,1,0,Security,reg oper spec account,block of flats,"Stone, brick",No
407706,Cash loans,F,N,N,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,1,0,0,0,Private service staff,0,0,0,0,0,0,Services,reg oper account,block of flats,"Stone, brick",No
318172,Cash loans,M,Y,N,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,1,0,0,0,Laborers,0,0,0,0,0,0,Business Entity Type 3,None,block of flats,Panel,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456186,Cash loans,M,N,N,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,1,1,0,0,Laborers,0,0,0,0,0,0,Business Entity Type 3,reg oper account,block of flats,"Stone, brick",No
456215,Cash loans,F,N,N,Unaccompanied,Working,Higher education,Civil marriage,House / apartment,1,1,0,0,Laborers,0,0,0,0,0,0,Construction,None,block of flats,Panel,No
456225,Cash loans,M,N,Y,Family,Working,Secondary / secondary special,Married,House / apartment,1,0,1,0,Managers,0,0,0,0,0,0,Business Entity Type 3,None,None,None,None


In [23]:
data.select_dtypes(exclude='object')

,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,OWN_CAR_AGE,CNT_FAM_MEMBERS,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,DAYS_BIRTH_1,DAYS_EMPLOYED_1,DAYS_REGISTRATION_1,DAYS_ID_PUBLISH_1,REGION_RATING_CLIENT_MEAN,WEEKDAY_APPR_PROCESS_START_cos,WEEKDAY_APPR_PROCESS_START_sin,HOUR_APPR_PROCESS_START_cos,HOUR_APPR_PROCESS_START_sin,EXT_SOURCE_AVG,DAYS_LAST_PHONE_CHANGE_1,number_documents_provided
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
371535,2,225000.0,208512.0,21496.5,180000.0,0.032561,3.0,4.0,0.1928,0.2148,0.9831,0.7756,0.0783,0.4264,0.2759,0.3471,0.3958,0.1093,0.0000,0.3014,0.0000,0.0646,0.1964,0.1477,0.9821,0.7648,0.0463,0.2014,0.1724,0.3333,0.3750,0.1010,0.0000,0.1340,0.0000,0.0000,0.1947,0.2280,0.9826,0.7786,0.0788,0.44,0.2759,0.3333,0.3958,0.1112,0.0000,0.3653,0.0000,0.0021,0.1575,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,2.0,33.564384,0.876712,16.915068,3.000000,1.0,-0.222521,-0.974928,-0.917211,-0.398401,0.665890,4.010959,1
269653,2,157500.0,582471.0,28458.0,409500.0,0.003122,0.0,4.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.00,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.249315,10.764384,15.873973,4.832877,3.0,-0.900969,0.433884,-0.576680,-0.816970,0.228414,0.424658,1
184406,1,180000.0,277969.5,8725.5,229500.0,0.009334,8.0,3.0,0.0912,0.0945,0.9796,0.7688,0.0190,0.0000,0.2069,0.1458,0.2083,0.0626,0.1051,0.0776,0.0000,0.0172,0.0546,0.0358,0.9762,0.7779,0.0191,0.0000,0.1379,0.1250,0.2083,0.0438,0.1148,0.0434,0.0000,0.0000,0.0921,0.0945,0.9796,0.7719,0.0191,0.00,0.2069,0.1458,0.2083,0.0637,0.1069,0.0790,0.0000,0.0175,0.0433,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40.895890,1.386301,19.904110,8.191781,2.0,-0.900969,0.433884,-0.775711,-0.631088,0.527127,0.767123,0
407706,1,360000.0,2013840.0,53253.0,1800000.0,0.022800,0.0,3.0,0.1402,0.1595,0.9771,0.6872,0.0070,0.0000,0.2759,0.1667,0.2083,0.0901,0.1126,0.1098,0.0077,0.0565,0.1429,0.1655,0.9772,0.6994,0.0071,0.0000,0.2759,0.1667,0.2083,0.0921,0.1230,0.1144,0.0078,0.0599,0.1416,0.1595,0.9771,0.6914,0.0071,0.00,0.2759,0.1667,0.2083,0.0916,0.1146,0.1118,0.0078,0.0577,0.1025,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,38.208219,0.671233,2.005479,15.854795,2.0,0.623490,-0.781831,-0.917211,-0.398401,0.712544,1.104110,2
318172,3,157500.0,912541.5,27796.5,693000.0,0.024610,11.0,5.0,0.0928,0.0945,0.9861,0.0000,0.0000,0.0000,0.2069,0.1667,0.0000,0.0152,0.0000,0.0874,0.0000,0.0000,0.0945,0.0981,0.9861,0.0000,0.0000,0.0000,0.2069,0.1667,0.0000,0.0155,0.0000,0.0911,0.0000,0.0000,0.0937,0.0945,0.9861,0.0000,0.0000,0.00,0.2069,0.1667,0.0000,0.0154,0.0000,0.0890,0.0000,0.0000,0.0920,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,34.539726,7.304110,17.758904,11.747945,2.0,-0.222521,0.974928,-0.334880,-0.942261,0.695231

In [24]:
def categorical_prep(data):
    from sklearn.preprocessing import OneHotEncoder

    # traitement des variables catégorielles
    
    # sélection du preprocessor (ici OneHot)
    categorical_transformer = OneHotEncoder(
        sparse=False, handle_unknown='ignore',dtype='int64')

    categ = categorical_transformer.fit_transform(data.select_dtypes('object'))

    # on transforme les données précédentes en DataFrame
    categ_dataf = pd.DataFrame(
        categ, columns=categorical_transformer.get_feature_names(data.select_dtypes('object').columns.tolist()),index = data.index)
    
    return categ_dataf

In [25]:
categorical_data = categorical_prep(data.select_dtypes('object'))

In [26]:
categorical_data.head()

,NAME_CONTRACT_TYPE_Cash loans,NAME_CONTRACT_TYPE_Revolving loans,CODE_GENDER_F,CODE_GENDER_M,FLAG_OWN_CAR_N,FLAG_OWN_CAR_Y,FLAG_OWN_REALTY_N,FLAG_OWN_REALTY_Y,NAME_TYPE_SUITE_Children,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Other,"NAME_TYPE_SUITE_Spouse, partner",NAME_TYPE_SUITE_Unaccompanied,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Working,NAME_EDUCATION_TYPE_Academic degree,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_FAMILY_STATUS_Civil marriage,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Widow,NAME_HOUSING_TYPE_Co-op apartment,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents,FLAG_EMP_PHONE_0,FLAG_EMP_PHONE_1,FLAG_WORK_PHONE_0,FLAG_WORK_PHONE_1,FLAG_PHONE_0,FLAG_PHONE_1,FLAG_EMAIL_0,FLAG_EMAIL_1,OCCUPATION_TYPE_Accountants,OCCUPATION_TYPE_Cleaning staff,OCCUPATION_TYPE_Cooking staff,OCCUPATION_TYPE_Core staff,OCCUPATION_TYPE_Drivers,OCCUPATION_TYPE_HR staff,OCCUPATION_TYPE_High skill tech staff,OCCUPATION_TYPE_IT staff,OCCUPATION_TYPE_Laborers,OCCUPATION_TYPE_Low-skill Laborers,OCCUPATION_TYPE_Managers,OCCUPATION_TYPE_Medicine staff,OCCUPATION_TYPE_Private service staff,OCCUPATION_TYPE_Realty agents,OCCUPATION_TYPE_Sales staff,OCCUPATION_TYPE_Secretaries,OCCUPATION_TYPE_Security staff,OCCUPATION_TYPE_Unemployed or Students,OCCUPATION_TYPE_Waiters/barmen staff,...,ORGANIZATION_TYPE_Industry: type 13,ORGANIZATION_TYPE_Industry: type 2,ORGANIZATION_TYPE_Industry: type 3,ORGANIZATION_TYPE_Industry: type 4,ORGANIZATION_TYPE_Industry: type 5,ORGANIZATION_TYPE_Industry: type 6,ORGANIZATION_TYPE_Industry: type 7,ORGANIZATION_TYPE_Industry: type 8,ORGANIZATION_TYPE_Industry: type 9,ORGANIZATION_TYPE_Insurance,ORGANIZATION_TYPE_Kindergarten,ORGANIZATION_TYPE_Legal Services,ORGANIZATION_TYPE_Medicine,ORGANIZATION_TYPE_Military,ORGANIZATION_TYPE_Mobile,ORGANIZATION_TYPE_Other,ORGANIZATION_TYPE_Pension,ORGANIZATION_TYPE_Police,ORGANIZATION_TYPE_Postal,ORGANIZATION_TYPE_Realtor,ORGANIZATION_TYPE_Religion,ORGANIZATION_TYPE_Restaurant,ORGANIZATION_TYPE_School,ORGANIZATION_TYPE_Security,ORGANIZATION_TYPE_Security Ministries,ORGANIZATION_TYPE_Self-employed,ORGANIZATION_TYPE_Services,ORGANIZATION_TYPE_Telecom,ORGANIZATION_TYPE_Trade: type 1,ORGANIZATION_TYPE_Trade: type 2,ORGANIZATION_TYPE_Trade: type 3,ORGANIZATION_TYPE_Trade: type 4,ORGANIZATION_TYPE_Trade: type 5,ORGANIZATION_TYPE_Trade: type 6,ORGANIZATION_TYPE_Trade: type 7,ORGANIZATION_TYPE_Transport: type 1,ORGANIZATION_TYPE_Transport: type 2,ORGANIZATION_TYPE_Transport: type 3,ORGANIZATION_TYPE_Transport: type 4,ORGANIZATION_TYPE_University,FONDKAPREMONT_MODE_None,FONDKAPREMONT_MODE_not specified,FONDKAPREMONT_MODE_org spec account,FONDKAPREMONT_MODE_reg oper account,FONDKAPREMONT_MODE_reg oper spec account,HOUSETYPE_MODE_None,HOUSETYPE_MODE_block of flats,HOUSETYPE_MODE_specific housing,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_None,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_None,EMERGENCYSTATE_MODE_Yes
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
371535,1,0,1,0,0,1,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0
269653,1,0,1,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...

In [27]:
def numerical_prep(data):
    from sklearn.preprocessing import StandardScaler

    # traitement des variables numériques
    
    # sélection du preprocessor (ici StandardScaler)
    numerical_transformer = StandardScaler()

    numeric = numerical_transformer.fit_transform(data.select_dtypes(exclude= 'object'))

    # on transforme les données précédentes en DataFrame
    num_dataf = pd.DataFrame(
        numeric, columns=data.columns,index = data.index)
    
    return num_dataf

In [28]:
numerical_data = numerical_prep(data.select_dtypes(exclude = 'object'))

In [29]:
numerical_data.head()

,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,OWN_CAR_AGE,CNT_FAM_MEMBERS,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,DAYS_BIRTH_1,DAYS_EMPLOYED_1,DAYS_REGISTRATION_1,DAYS_ID_PUBLISH_1,REGION_RATING_CLIENT_MEAN,WEEKDAY_APPR_PROCESS_START_cos,WEEKDAY_APPR_PROCESS_START_sin,HOUR_APPR_PROCESS_START_cos,HOUR_APPR_PROCESS_START_sin,EXT_SOURCE_AVG,DAYS_LAST_PHONE_CHANGE_1,number_documents_provided
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
371535,2.194833,0.575344,-0.971217,-0.387662,-0.969613,0.855914,-0.123369,2.029623,1.412636,2.618485,0.990515,1.459307,1.408833,3.905844,1.967716,1.539869,2.278167,1.359622,-0.451582,2.624364,-0.101403,1.066172,1.482942,1.622533,0.989176,1.410943,0.746876,1.710373,0.992846,1.480182,2.160949,1.233767,-0.449196,0.858285,-0.095794,-0.246054,1.421177,2.824393,0.989368,1.458927,1.417175,4.055862,1.970563,1.449358,2.276931,1.369140,-0.451516,3.248249,-0.100054,-0.213441,1.132119,-0.179548,1.917893,-0.173773,2.483107,-0.071837,-0.057811,-0.154684,-0.26990,0.948989,0.122301,-0.857503,-0.708011,0.344432,-1.249601,-2.094332,-0.108177,-1.389176,-0.654184,-0.470933,1.061026,0.615543,0.202334
269653,2.194833,-0.107180,-0.037470,0.094991,-0.345241,-1.284388,-0.454128,2.029623,-0.598099,-0.532566,-1.013424,-0.693578,-0.287600,-0.366665,-0.718447,-0.740067,-0.522774,-0.439676,-0.451582,-0.560989,-0.101403,-0.259043,-0.588363,-0.521979,-1.012618,-0.694413,-0.281522,-0.354342,-0.704762,-0.736226,-0.519393,-0.431152,-0.449196,-0.551401,-0.095794,-0.246054,-0.596521,-0.531630,-1.013317,-0.693920,-0.286972,-0.363656,-0.716179,-0.738479,-0.521844,-0.439991,-0.451516,-0.559634,-0.100054,-0.256250,-0.567955,-0.610889,-0.324748,-0.608820,-0.279141,-0.071837,-0.057811,-0.154684,-0.26990,-0.281582,-1.006836,-0.967509,0.868694,0.236318,-0.806647,1.905434,-1.150135,0.510710,0.212536,-1.133037,-1.816555,-0.972075,0.202334
184406,0.807610,0.120328,-0.797787,-1.273098,-0.834944,-0.832758,0.427895,0.930848,0.353037,0.853720,0.983381,1.440432,0.124051,-0.366665,1.295932,0.217624,0.951294,0.590846,1.052857,0.259128,-0.101403,0.093800,-0.012532,-0.002185,0.977150,1.447005,0.142720,-0.354342,0.653127,0.095010,0.969448,0.290862,1.110312,-0.094831,-0.095794,-0.246054,0.357922,0.859353,0.983254,1.440401,0.126089,-0.363656,1.298634,0.218577,0.951083,0.596354,1.060396,0.263861,-0.100054,0.100494,-0.100570,-0.610889,-0.324748,-0.608820,-0.279141,-0.071837,-0.057811,-0.154684,-0.26990,-0.281582,-1.006836,-0.244223,-0.626751,0.654835,0.005105,-0.094449,-1.150135,0.510710,-0.294038,-0.839004,0.148287,-0.820469,-2.721514
407706,0.807610,1.940390,3.536547,1.814072,3.437715,0.146260,-0.454128,0.930848,0.864064,1.807251,0.978285,1.213930,-0.135939,-0.366665,1.967716,0.354907,0.951294,1.043552,1.160215,0.599435,0.192367,0.900008,0.918712,1.880978,0.979188,1.230909,-0.123819,-0.354342,2.012001,0.372310,0.969448,1.087056,1.221706,0.652092,0.208941,0.978786,0.870896,1.816114,0.978158,1.217817,-0.133426,-0.3

In [30]:
display(target.shape,categorical_data.shape, numerical_data.shape )

(207222,)

(207222, 150)

(207222, 73)

In [31]:
data = pd.concat([target, categorical_data,numerical_data], axis=1)

In [32]:
data.shape

(207222, 224)

In [33]:
data.to_csv('data_prep.csv', sep='\t')